# QUESTION 1 : Get Data

In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import requests

In [15]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

tables = pd.read_html(r.text)

df=pd.DataFrame(tables[0])

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df.columns=['Postcode','Borough','Neighbourhood']

df.drop([0],axis=0,inplace=True)

df.reset_index()

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, 
# you will notice that M5A is listed twice and has two neighborhoods: 
# Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods 
# separated with a comma as shown in row 11 in the above table.

df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))



In [16]:
# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, 
# the value of the Borough and the Neighborhood columns will be Queen's Park.

df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']

#df2.head()

In [18]:
df1.head(20)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
M1L,Scarborough,"Oakridge,Golden Mile,Clairlea"
M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest"


In [20]:
df2 = df1.reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
#Now we can remove the duplicate boroughts as follows:
df2['Borough']= df2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [23]:
df2.head()
df2.shape

(103, 3)

# QUESTION 2:  Getting the geo coordinates

In [31]:
pip install geopy


The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [32]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


latitude is :- 51.5073219 
longtitude is:- -0.1276474


In [33]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 112665056, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [34]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

from geopy.geocoders import Nominatim
geolocator = Nominatim()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [38]:
df_geopy1 = df2
#df_geopy1

In [39]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

df_geopy1['address'] = df2[['Postcode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


,Postcode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Malvern,Rouge","M1B, Scarborough, Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,Scarborough,"Morningside,West Hill,Guildwood","M1E, Scarborough, Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [40]:
df_geopy1.shape

(103, 4)

In [41]:
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [42]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)
#df_geopy1
# code holds true up until i=102
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [43]:
#df_geopy1.head()

In [44]:
df_geopy1.shape

(103, 4)

In [45]:
df_geopy1.to_csv('geopy1.csv')

In [46]:
#testing data
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1G, Scarborough, Woburn")


print(location.address)

print((location.latitude, location.longitude))

print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada
(43.7598243, -79.2252908)
{'place_id': 4972888, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 558715617, 'boundingbox': ['43.7597743', '43.7598743', '-79.2253408', '-79.2252408'], 'lat': '43.7598243', 'lon': '-79.2252908', 'display_name': 'Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada', 'class': 'place', 'type': 'neighbourhood', 'importance': 0.558803667479001}


In [47]:
pip install geocoder


The following command must be run outside of the IPython shell:

    $ pip install geocoder

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [49]:
df2.to_csv('geopy.csv')

In [50]:
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     #for row in reader:
         #print(row['Postcode'],row['Borough'], row['Neighbourhood'] )

In [51]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1B Scarborough Rouge,Malvern")

print(location.address)

print((location.latitude, location.longitude))

print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada
(43.8091955, -79.2217008)
{'place_id': 4979177, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 558715616, 'boundingbox': ['43.8091455', '43.8092455', '-79.2217508', '-79.2216508'], 'lat': '43.8091955', 'lon': '-79.2217008', 'display_name': 'Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada', 'class': 'place', 'type': 'neighbourhood', 'importance': 0.769448365439565}


In [52]:
pip install geopy


The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [53]:
from geopy.geocoders import Nominatim
print('Nominatim imported')

Nominatim imported


In [57]:
df_geopy.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront


In [58]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']

In [59]:
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods"
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village"
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront"


In [60]:
nom = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [61]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods","(Parkwoods Village Drive, Parkway East, Don Va..."
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village",None
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront",None


In [62]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M3A,North York,Parkwoods,"M3A,North York,Parkwoods","(Parkwoods Village Drive, Parkway East, Don Va...",43.761224,-79.323986
1,M4A,North York,Victoria Village,"M4A,North York,Victoria Village",None,NaN,NaN
2,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto,Harbourfront",None,NaN,NaN


In [63]:
df_geopy.to_csv('geo_loc_py.csv')

In [64]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])


The latitude of M3A,North York,Parkwoods is 43.7612239 and its longitude is -79.3239857


In [65]:
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data2 = pd.read_csv("geopy.csv") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,address
0,0,M1B,Scarborough,"Malvern,Rouge","M1B, Scarborough, Malvern,Rouge"
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,2,M1E,Scarborough,"Morningside,West Hill,Guildwood","M1E, Scarborough, Morningside,West Hill,Guildwood"
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [73]:
data3 = pd.read_csv("http://cocl.us/Geospatial_data") 
# Preview the first 5 lines of the loaded data 
data3.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [75]:
data3.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
data3.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [76]:
data1 = pd.merge(data3, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data1.head()

,Postcode,Latitude,Longitude,Unnamed: 0,Borough,Neighbourhood,address
0,M1B,43.806686,-79.194353,0,Scarborough,"Malvern,Rouge","M1B, Scarborough, Malvern,Rouge"
1,M1C,43.784535,-79.160497,1,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,43.763573,-79.188711,2,Scarborough,"Morningside,West Hill,Guildwood","M1E, Scarborough, Morningside,West Hill,Guildwood"
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [77]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 7 columns):
Postcode         103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
Unnamed: 0       103 non-null int64
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 6.4+ KB


In [78]:
cols = data1.columns.tolist()
cols

['Postcode',
 'Latitude',
 'Longitude',
 'Unnamed: 0',
 'Borough',
 'Neighbourhood',
 'address']

In [79]:
new_column_order = ['Postcode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order

['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [80]:
data1 = data1[new_column_order]
sorted_df = data1.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])

In [81]:
sorted_df.reset_index(inplace=True)
sorted_df.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,12,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,89,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,68,M5V,DowntownToronto,"Bathurst Quay,Harbourfront West,Island airport...",43.628947,-79.394420
3,19,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,56,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306


In [82]:
sorted_cols =sorted_df.columns.tolist()
new_column_order2 = ['Postcode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order2

['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [83]:
sorted_dataframe = sorted_df[new_column_order]
sorted_dataframe.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484
2,M5V,DowntownToronto,"Bathurst Quay,Harbourfront West,Island airport...",43.628947,-79.394420
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5E,DowntownToronto,Berczy Park,43.644771,-79.373306


In [84]:
sorted_dataframe.to_csv('sorted_geoloc.csv')